In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
#create new train dataset from valid predictions
df = pd.read_csv('../input/train10fold/train-folds (1).csv')
df_test = pd.read_csv('../input/30-days-of-ml/test.csv')

df1 = pd.read_csv('../input/lavel1blendingdataset/valid_pred1.csv')
df1.columns = ['id', 'pred1']
df2 = pd.read_csv('../input/lavel1blendingdataset/valid_pred2.csv')
df2.columns = ['id', 'pred2']
df3 = pd.read_csv('../input/lavel1blendingdataset/valid_pred3.csv')
df3.columns = ['id', 'pred3']
df4 = pd.read_csv('../input/lavel1blendingdataset/valid_pred4.csv')
df4.columns = ['id', 'pred4']
df5 = pd.read_csv('../input/lavel1blendingdataset/valid_pred5.csv')
df5.columns = ['id', 'pred5']

df_test1 = pd.read_csv('../input/lavel1blendingdataset/test_pred1.csv')
df_test1.columns = ['id', 'pred1']
df_test2 = pd.read_csv('../input/lavel1blendingdataset/test_pred2.csv')
df_test2.columns = ['id', 'pred2']
df_test3 = pd.read_csv('../input/lavel1blendingdataset/test_pred3.csv')
df_test3.columns = ['id', 'pred3']
df_test4 = pd.read_csv('../input/lavel1blendingdataset/test_pred4.csv')
df_test4.columns = ['id', 'pred4']
df_test5 = pd.read_csv('../input/lavel1blendingdataset/test_pred5.csv')
df_test5.columns = ['id', 'pred5']

df = df.merge(df1, on='id', how='left')
df = df.merge(df2, on='id', how='left')
df = df.merge(df3, on='id', how='left')
df = df.merge(df4, on='id', how='left')
df = df.merge(df5, on='id', how='left')

df_test = df_test.merge(df_test1, on='id', how='left')
df_test = df_test.merge(df_test2, on='id', how='left')
df_test = df_test.merge(df_test3, on='id', how='left')
df_test = df_test.merge(df_test4, on='id', how='left')
df_test = df_test.merge(df_test5, on='id', how='left')

df.head()


In [ ]:
submission_data = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

features = ['pred1', 'pred2', 'pred3', 'pred4', 'pred5']

xtest = df_test[features]
xtest = xtest.copy()

final_valid_preds = {}
final_test_preds = []
rmse_score = []
for fold in range(10):
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    
    valid_ids = xvalid.id.values.tolist() # to keep all the ids of the validation data
        
    ytrain = xtrain.target
    yvalid = xvalid.target
        
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    best_params = {'learning_rate': 0.07853392035787837,
                     'colsample_bytree': 0.170759104940733,
                     'max_depth': 3,
                     'reg_lambda': 1.7549293092194938e-05,
                     'reg_alpha': 14.68267919457715,
                     'subsample': 0.8031450486786944,
                     'alpha': 30
                  }
    model = XGBRegressor(objective='reg:squarederror',
                         n_estimators=5000,
                         random_state=0,
                         n_jobs = 4,
                         **best_params,
                         tree_method='gpu_hist',
                         gpu_id=0, 
                         predictor='gpu_predictor'
                        )

    model.fit(xtrain, ytrain, 
              early_stopping_rounds=300, 
              eval_set=[(xvalid, yvalid)], 
              verbose=1000)
    
    valid_preds = model.predict(xvalid)
    test_preds = model.predict(xtest)
    
    final_valid_preds.update(dict(zip(valid_ids, valid_preds)))
    final_test_preds.append(test_preds)

    rmse = mean_squared_error(yvalid, valid_preds, squared=False)
    rmse_score.append(rmse)
    print(fold, rmse)
    
print(np.mean(rmse_score), np.std(rmse_score))

final_valid_preds = pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_preds.columns = ['id', 'pred1']
final_valid_preds.to_csv('lavel1_valid_pred1.csv', index=False)

submission_data.target = np.mean(np.column_stack(final_test_preds), axis = 1)
submission_data.columns = ['id', 'pred1']
submission_data.to_csv('level1_test_pred1.csv', index=False)

In [ ]:
submission_data = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

features = ['pred1', 'pred2', 'pred3', 'pred4', 'pred5']

xtest = df_test[features]
xtest = xtest.copy()

final_valid_preds = {}
final_test_preds = []
rmse_score = []
for fold in range(10):
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    
    valid_ids = xvalid.id.values.tolist() # to keep all the ids of the validation data
        
    ytrain = xtrain.target
    yvalid = xvalid.target
        
    xtrain = xtrain[features]
    xvalid = xvalid[features]

    model = RandomForestRegressor(n_estimators = 500, 
                                 n_jobs = -1, 
                                 max_depth=3)

    model.fit(xtrain, ytrain)
    
    valid_preds = model.predict(xvalid)
    test_preds = model.predict(xtest)
    
    final_valid_preds.update(dict(zip(valid_ids, valid_preds)))
    final_test_preds.append(test_preds)

    rmse = mean_squared_error(yvalid, valid_preds, squared=False)
    rmse_score.append(rmse)
    print(fold, rmse)
    
print(np.mean(rmse_score), np.std(rmse_score))

final_valid_preds = pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_preds.columns = ['id', 'pred2']
final_valid_preds.to_csv('lavel1_valid_pred2.csv', index=False)

submission_data.target = np.mean(np.column_stack(final_test_preds), axis = 1)
submission_data.columns = ['id', 'pred2']
submission_data.to_csv('level1_test_pred2.csv', index=False)

In [ ]:
submission_data = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

features = ['pred1', 'pred2', 'pred3', 'pred4', 'pred5']

xtest = df_test[features]
xtest = xtest.copy()

final_valid_preds = {}
final_test_preds = []
rmse_score = []
for fold in range(10):
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    
    valid_ids = xvalid.id.values.tolist() # to keep all the ids of the validation data
        
    ytrain = xtrain.target
    yvalid = xvalid.target
        
    xtrain = xtrain[features]
    xvalid = xvalid[features]

    model = GradientBoostingRegressor(n_estimators = 500, 
                                 max_depth=3)

    model.fit(xtrain, ytrain)
    
    valid_preds = model.predict(xvalid)
    test_preds = model.predict(xtest)
    
    final_valid_preds.update(dict(zip(valid_ids, valid_preds)))
    final_test_preds.append(test_preds)

    rmse = mean_squared_error(yvalid, valid_preds, squared=False)
    rmse_score.append(rmse)
    print(fold, rmse)
    
print(np.mean(rmse_score), np.std(rmse_score))

final_valid_preds = pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_preds.columns = ['id', 'pred3']
final_valid_preds.to_csv('lavel1_valid_pred3.csv', index=False)
3
submission_data.target = np.mean(np.column_stack(final_test_preds), axis = 1)
submission_data.columns = ['id', 'pred3']
submission_data.to_csv('level1_test_pred3.csv', index=False)

In [ ]:
import lightgbm as lgb
submission_data = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

features = ['pred1', 'pred2', 'pred3', 'pred4', 'pred5']

xtest = df_test[features]
xtest = xtest.copy()

final_valid_preds = {}
final_test_preds = []
rmse_score = []
for fold in range(10):
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    
    valid_ids = xvalid.id.values.tolist() # to keep all the ids of the validation data
        
    ytrain = xtrain.target
    yvalid = xvalid.target
        
    xtrain = xtrain[features]
    xvalid = xvalid[features]

    best_params = {'learning_rate': 0.07853392035787837,
                     'colsample_bytree': 0.170759104940733,
                     'max_depth': 3,
                     'reg_lambda': 1.7549293092194938e-05,
                     'reg_alpha': 14.68267919457715,
                     'subsample': 0.8031450486786944,
                     'alpha': 30
                  }
    model = lgb.LGBMRegressor(n_estimators=5000,
                         random_state=0,
                         **best_params,
                         device='gpu'
                             )

    model.fit(xtrain, ytrain, 
              early_stopping_rounds=300, 
              eval_set=[(xvalid, yvalid)], 
              verbose=1000)
    
    valid_preds = model.predict(xvalid)
    test_preds = model.predict(xtest)
    
    final_valid_preds.update(dict(zip(valid_ids, valid_preds)))
    final_test_preds.append(test_preds)

    rmse = mean_squared_error(yvalid, valid_preds, squared=False)
    rmse_score.append(rmse)
    print(fold, rmse)
    
print(np.mean(rmse_score), np.std(rmse_score))

final_valid_preds = pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_preds.columns = ['id', 'pred4']
final_valid_preds.to_csv('lavel1_valid_pred4.csv', index=False)

submission_data.target = np.mean(np.column_stack(final_test_preds), axis = 1)
submission_data.columns = ['id', 'pred4']
submission_data.to_csv('level1_test_pred4.csv', index=False)

In [ ]:
#create new train dataset from valid predictions
df = pd.read_csv('../input/train10fold/train-folds (1).csv')
df_test = pd.read_csv('../input/30-days-of-ml/test.csv')
submission_data = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')


df1 = pd.read_csv('./lavel1_valid_pred1.csv')
df2 = pd.read_csv('./lavel1_valid_pred2.csv')
df3 = pd.read_csv('./lavel1_valid_pred3.csv')
df4 = pd.read_csv('./lavel1_valid_pred4.csv')

df_test1 = pd.read_csv('./level1_test_pred1.csv')
df_test2 = pd.read_csv('./level1_test_pred2.csv')
df_test3 = pd.read_csv('./level1_test_pred3.csv')
df_test4 = pd.read_csv('./level1_test_pred4.csv')

df = df.merge(df1, on='id', how='left')
df = df.merge(df2, on='id', how='left')
df = df.merge(df3, on='id', how='left')
df = df.merge(df4, on='id', how='left')

df_test = df_test.merge(df_test1, on='id', how='left')
df_test = df_test.merge(df_test2, on='id', how='left')
df_test = df_test.merge(df_test3, on='id', how='left')
df_test = df_test.merge(df_test4, on='id', how='left')

df.head()


In [ ]:
useful_features = ['pred1', 'pred2', 'pred3', 'pred4']

xtest = df_test.copy()
xtest = xtest[useful_features]

final_preds = []
rmse_score = []
for fold in range(10):
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    
    ytrain = xtrain.target
    yvalid = xvalid.target
        
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = LinearRegression()
    model.fit(xtrain, ytrain)
    
    valid_preds = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_preds.append(test_preds)

    rmse = mean_squared_error(yvalid, valid_preds, squared=False)
    rmse_score.append(rmse)
    print(fold, rmse)
    
print(np.mean(rmse_score), np.std(rmse_score))

In [ ]:
submission_data.target = np.mean(np.column_stack(final_preds), axis = 1)
submission_data.to_csv('submission.csv', index=False)